# Project 1: Reading comprehension

Changes to stage 1 in difficult to spot parts are highlighted with <span style="color:blue">blue</span>.


# Introduction

Does the given text answer the given question. This is a relativly simple task for humans. But for NLP models it can be hard.

Therefore a benchmark exists: `BoolQ`.
A Wikipedia passage is given with `passage` and a `question`. `answer` is either `True` or `False` depending on if the `question` is answered by the `passage`. 

A classification model is created with 2 layers and ReLu activation between them. A sigmoid is employed as the final activation, as we are dealing with a binary problem.

For the final model multiple experiments were made. Different optimizers (Adam, AdamW), loss functions (Binary Cross Entropy, Hinge Embedding) and learning rates (1e-7 - 1e-3) were tried.
The chosen parameter for the final model are `AdamW`, `BCELoss` and `2e-4` as they had the best performance.

The result is a model with an accuracy of 61.65%, F1 75.1%, AUC-ROC 54.7%.

W&B Link: https://wandb.ai/yelin-zhang-hslu/project-1-boolq/workspace?nw=2448or6ijis

# Setup

## Dependencies
Install all necessary dependencies
- PyTorch: `torch lightning`
- Hugging Face: `huggingface_hub datasets`
- Weights & Biases: `wandb`
- nltk: `nltk`
- fasttext: `fasttext`
- numpy: `numpy`
- optuna: `optuna optuna-integration[pytorch_lightning]`

Optional
- Lint and Formatting: `ruff`

Dependencies are pinned to the version the code was created with. 

## Tools used
- GPUHub JupyterLab
- Pytorch Lightning documentation

## Notebook setup
Log into Hugging Face and Weights & Biases.

In [1]:
%pip install torch lightning huggingface_hub datasets wandb nltk fasttext numpy optuna optuna-integration[pytorch_lightning] ruff

Note: you may need to restart the kernel to use updated packages.


In [30]:
from datasets import load_dataset
from pathlib import Path
import subprocess
from huggingface_hub import login, hf_hub_download
from datasets import Value
import wandb
from lightning.pytorch.loggers import WandbLogger
from torch import optim, nn, utils
import lightning as L
import nltk
import torch
import fasttext
import numpy as np
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from torchmetrics.classification import BinaryF1Score, BinaryAUROC, BinaryAccuracy

L.pytorch.seed_everything(42, workers=True)
subprocess.run(["ruff", "check", "--fix", "."])
subprocess.run(["ruff", "format", "."])
WANDB_PROJECT = "project-1-boolq"

Seed set to 42


All checks passed!
3 files left unchanged


In [3]:
login()

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yelin-zhang (yelin-zhang-hslu). Use `wandb login --relogin` to force relogin


True

# Preprocessing

Predefined requirements:
- Train / Validation / Test split
- Existing word embedding model: word2vec, GloVe, fastText

Download the BoolQ dataset with `datasets` and split it in the predefined way.

Data treatment steps:
- Tokenize
- Lower case
- Stop word removal
- Lemmatization 
- truncating of passage (enforce maximum length, ~99% not truncated)
- Embedding with fasttext
- <span style="color:blue">padding of too short question and passage vectors</span>

Used features: `question`, `answer`, `passage` (all of them)

Input format: `question` and `passage` vectors  <span style="color:blue">concatenated into a large query vector</span>.
Label format: `answer` 1 or 0

Batching: None, dataset is small enough (<span style="color:blue">8</span>.5k rows in train)

Correctness tests:
- Check texts before embedding if they still make sense 
- <span style="color:blue">Check shape of embedding vectors</span>

## Implementaton
First lowercase the sentence and remove non ascii characters. This is done to remove pheonetics and other special scripts which usually are included in Wikipedia articles.

Preprocess the dataset with nltk to tokenize and lemmatize the words.
I used lemmatization instead of stemming because stemmed words are not real words.

The processing is done with the built in method of `map` from the HuggingFace dataset class.
This improves the speed of the processing and displays built in progress bars.

An additional speed gain is through `with_format("np")`, calling this method converts all lists in the dataset to `np.array`.
By not having to convert the datatype in the function we also save some time.

The tokenized and lemmatized words are then sent to `fasttext` to generate word vectors.

Truncating and padding is done in a second step, after all passage lengths have been gathered.
The 99% longest passage is used as a cutoff. Anything longer than that will just be truncated, as I assume the desired information should be in the first few sentences.
Questions are always shorter therfore they only have to be padded to the maximum length to be able to concatenate `question` and `passage` into single input: `query` which the model processes.

In [4]:
# https://github.com/nltk/nltk Bird, S., Klein, E., & Loper, E. (2009). Natural Language Processing with Python: Analyzing Text with the Natural Language Toolkit. O'Reilly Media, Inc.. https://www.nltk.org/book/
nltk.download("stopwords")
nltk.download("wordnet")
# https://huggingface.co/facebook/fasttext-en-vectors: A. Joulin, E. Grave, P. Bojanowski, T. Mikolov, Bag of Tricks for Efficient Text Classification
model_path = hf_hub_download(
    repo_id="facebook/fasttext-en-vectors", filename="model.bin"
)

# Predefined dataset loading code
train_raw = load_dataset("google/boolq", split="train[:-1000]")
valid_raw = load_dataset("google/boolq", split="train[-1000:]")
test_raw = load_dataset("google/boolq", split="validation")

print(len(train_raw), len(valid_raw), len(test_raw))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


8427 1000 3270


In [5]:
stop_words = set(nltk.corpus.stopwords.words("english"))
stop_words = stop_words | {'"', "'", "''", "`", "``", "'s"}
embedding_model = fasttext.load_model(model_path)
lemmatizer = nltk.stem.WordNetLemmatizer()

text_values = ["question", "passage"]
passage_lens = []


def preprocess_dataset(row, debug=False):
    for key in text_values:
        tokenized_value = [
            word
            for word in nltk.word_tokenize(
                row[key].lower().encode("ascii", errors="ignore").decode()
            )
            if word not in stop_words
        ]
        lemmatized_value = [lemmatizer.lemmatize(word) for word in tokenized_value]
        embedded_value = [embedding_model[word] for word in lemmatized_value]
        row[key] = embedded_value
        if debug:
            print(
                f"{key}:\ntokenized: {tokenized_value}\nlemmatized: {lemmatized_value}"
            )
    passage_lens.append(len(row["passage"]))
    return row


preprocess_dataset(train_raw[0], True)

train_processed = (
    train_raw.map(preprocess_dataset)
    .cast_column("answer", Value("int32"))
    .with_format("np")
)
valid_processed = (
    valid_raw.map(preprocess_dataset)
    .cast_column("answer", Value("int32"))
    .with_format("np")
)
test_processed = (
    test_raw.map(preprocess_dataset)
    .cast_column("answer", Value("int32"))
    .with_format("np")
)

assert train_processed[0]["answer"] == 1, "Answer should be 1"
assert (
    train_processed[0]["passage"].shape[1] == 300
), "Fasttext word vecor should be 300 long"

max_len = int(np.percentile(passage_lens, 99))


def truncate_pad_data(row):
    for key in text_values:
        p = row[key]
        if p.shape[0] > max_len:
            p = p[:max_len]
        elif p.shape[0] < max_len:
            p = np.pad(p, ((0, max_len - p.shape[0]), (0, 0)))
        row[key] = p
    row["query"] = np.concatenate((row["question"], p))
    return row


train = train_processed.map(truncate_pad_data).remove_columns(["question", "passage"])
valid = valid_processed.map(truncate_pad_data).remove_columns(["question", "passage"])
test = test_processed.map(truncate_pad_data).remove_columns(["question", "passage"])

assert train[0]["query"].shape == (
    max_len * 2,
    300,
), f"Query shape should be {max_len*2}, 300"
assert "question" not in train[0], "question key should not be in final dataset"
assert "passage" not in train[0], "passage key should not be in final dataset"

Parameter 'function'=<function preprocess_dataset at 0x7fba3b20bc40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


question:
tokenized: ['iran', 'afghanistan', 'speak', 'language']
lemmatized: ['iran', 'afghanistan', 'speak', 'language']
passage:
tokenized: ['persian', '(', '/prn', ',', '-n/', ')', ',', 'also', 'known', 'endonym', 'farsi', '(', 'frsi', '(', 'fsi', ')', '(', 'listen', ')', ')', ',', 'one', 'western', 'iranian', 'languages', 'within', 'indo-iranian', 'branch', 'indo-european', 'language', 'family', '.', 'primarily', 'spoken', 'iran', ',', 'afghanistan', '(', 'officially', 'known', 'dari', 'since', '1958', ')', ',', 'tajikistan', '(', 'officially', 'known', 'tajiki', 'since', 'soviet', 'era', ')', ',', 'regions', 'historically', 'persianate', 'societies', 'considered', 'part', 'greater', 'iran', '.', 'written', 'persian', 'alphabet', ',', 'modified', 'variant', 'arabic', 'script', ',', 'evolved', 'aramaic', 'alphabet', '.']
lemmatized: ['persian', '(', '/prn', ',', '-n/', ')', ',', 'also', 'known', 'endonym', 'farsi', '(', 'frsi', '(', 'fsi', ')', '(', 'listen', ')', ')', ',', 'one', 

Map:   0%|          | 0/8427 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/8427 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3270 [00:00<?, ? examples/s]

Map:   0%|          | 0/8427 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

# Model

Predefined requirements:
- Classifier
    - 2 Layers
    - ReLu

network architecture

- input layer
    - dim: <span style="color:blue">fasttext vector length x 1</span>
    - ReLu activation
- ~~<span style="color:blue">2 hidden layers</span>~~
- output layer
    - output of class (1 true, 0 false)
        - dim: <span style="color:blue">max length of input vectors x 1</span>
        - ~~<span style="color:blue">not probability output because the question should be answered with yes or no, not 60% yes</span>~~
    - sigmoid activation
- normalization: done in preprocessing
- regularization: done by optimizer

Loss function either:
- HingeEmbeddingLoss: measures whether two inputs are similar or not
- Binary Cross-Entropy: separate to classes

Optimizer either:
- Adam: good default choice
- AdamW: supposed improvements to Adam

Experiments:
- Different Loss and Optimizer combiniations
- ~~<span style="color:blue">Size of input & hidden layers</span>~~
- ~~<span style="color:blue">Epochs</span>~~, Learning rate 

Checkpoints: every few epochs 
early stops: if <span style="color:blue">validation accuracy</span> does not improve

correctness test:
<span style="color:blue">training, validation and test run with 1 example, by setting</span> `fast_dev_run` 

In [10]:
class QClassifier(L.LightningModule):
    def __init__(self, loss=torch.nn.BCELoss, optimizer=optim.Adam, lr=1e-3):
        super().__init__()
        self.lr = lr
        self.loss = loss()
        self.optimizer = optimizer
        self.accuracy = BinaryAccuracy()
        self.f1 = BinaryF1Score()
        self.auroc = BinaryAUROC()

        self.layer1 = nn.Linear(300, 1)
        self.layer2 = nn.Linear(max_len * 2, 1)

        self.save_hyperparameters()

    def configure_optimizers(self):
        return self.optimizer(self.parameters(), lr=self.lr)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.squeeze(x)
        x = torch.sigmoid(self.layer2(x))
        return x

    def training_step(self, batch, batch_idx):
        _, loss, acc, f1_score, auc_score = self._get_pred_loss_accuracy(batch)
        self.log("train_loss", loss)
        self.log("train_accuracy", acc)
        self.log("train_f1", f1_score)
        self.log("train_auc", auc_score)
        return loss

    def validation_step(self, batch, batch_idx):
        preds, loss, acc, f1_score, auc_score = self._get_pred_loss_accuracy(batch)
        self.log("val_loss", loss)
        self.log("val_accuracy", acc)
        self.log("val_f1", f1_score)
        self.log("val_auc", auc_score)
        return preds

    def test_step(self, batch, batch_idx):
        preds, loss, acc, f1_score, auc_score = self._get_pred_loss_accuracy(batch)
        self.log("test_accuracy", acc)
        self.log("test_f1", f1_score)
        self.log("test_auc", auc_score)
        return preds

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch["query"])

    def _get_pred_loss_accuracy(self, batch):
        answers = batch["answer"].float()
        pred = self.forward(batch["query"])
        pred = torch.squeeze(pred)
        loss = self.loss(pred, answers)
        acc = self.accuracy(pred, answers)
        f1_score = self.f1(pred, answers)
        auc_score = self.auroc(pred, answers)
        return pred, loss, acc, f1_score, auc_score


trainer = L.Trainer(fast_dev_run=True, limit_test_batches=1)
m = QClassifier()
trainer.fit(
    model=m,
    train_dataloaders=utils.data.DataLoader(train, batch_size=2),
    val_dataloaders=utils.data.DataLoader(train, batch_size=2),
)
trainer.test(model=m, dataloaders=utils.data.DataLoader(train, batch_size=2))
trainer.predict(model=m, dataloaders=utils.data.DataLoader(train, batch_size=2))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type           | Params | Mode 
----------------------------------------------------
0 | loss     | BCELoss        | 0      | train
1 | accuracy | BinaryAccuracy | 0      | train
2 | f1       | BinaryF1Score  | 0      | train
3 | auroc    | BinaryAUROC    | 0      | train
4 | layer1   | Linear         | 301    | train
5 | layer2   | Linear         | 391    | train
----------------------------------------------------
692       Trainable params
0         Non-trainable params
692       Total params
0.003     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy                 1.0
        test_auc                    0.0
         test_f1                    1.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[0.5047],
         [0.5010]])]

# Training

Initialize Weights & Biases project for the project

Use k fold cross validation to avoid overfitting

1. Define experiement with different hyperparameters
2. Train model with train dataset split
3. Check model performance with validation dataset split
4. Log training run to Weights & Biases
5. Repeat

After all experiments have run select best runs hyperparameters for the final model.

## Implementation
k fold cross validation was not needed, as I did not manage to overfit the model. It seems to always be underfit.

Model checkpoints are created every epoch and saved to wandb. Which can be retrieved later.

The training is done with automatic hyperparameter sweeps by `optuna`. Optimizer, Loss function and learning rate are automatically chosen and tried.
20 sweeps were chosen as there are only 4 discrete options and the learning rate.

Pruning is also done by optuna, based on the validation accuracy.
If the accuracy at the current step is better than the median of previous run at the same step.

## Problems
I was not able to use the `deterministic` parameter in the `Trainer` class, as it produced cuda errors which I was not able to resolve.

In [11]:
train_loader = utils.data.DataLoader(
    train, batch_size=32, num_workers=2, pin_memory=True
)
valid_loader = utils.data.DataLoader(
    valid, batch_size=32, num_workers=2, pin_memory=True
)


def objective(trial):
    _optimizer = trial.suggest_categorical("optimizer", ["Adam", "AdamW"])
    _loss = trial.suggest_categorical("loss", ["BCELoss", "HingeEmbeddingLoss"])
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True)

    loss = getattr(torch.nn, _loss)
    optimizer = getattr(torch.optim, _optimizer)

    wandb.init(project=WANDB_PROJECT)
    logger = WandbLogger(project=WANDB_PROJECT, log_model="all")
    classifier = QClassifier(optimizer=optimizer, loss=loss, lr=learning_rate)
    trainer = L.Trainer(
        max_epochs=50,
        logger=logger,
        accelerator="gpu",
        devices=1,
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_accuracy")],
    )
    trainer.fit(
        model=classifier, train_dataloaders=train_loader, val_dataloaders=valid_loader
    )
    trial.set_user_attr(key="model", value=classifier)
    trial.set_user_attr(key="wandb_id", value=wandb.run.id)

    logger.finalize("success")
    return trainer.callback_metrics["val_accuracy"].item()


study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)

print("Number of finished trials:", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value:", trial.value)
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-10-06 15:48:58,915] A new study created in memory with name: no-name-dd8724fc-304e-4fc0-a4b2-9e50f9e4b006


epoch,▁▁▁▁▁▁
train_accuracy,▅██▁▇
train_auc,▁▂▁█▅
train_f1,▁▁▁▁▁
train_loss,▄▁▁█▂
trainer/global_step,▁▃▄▆██
val_accuracy,▁
val_auc,▁
val_f1,▁
val_loss,▁
epoch,0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type           | Params | Mode 
----------------------------------------------------
0 | loss     | BCELoss        | 0      | train
1 | accuracy | BinaryAccuracy | 0      | train
2 | f1       | BinaryF1Score  | 0      | train
3 | auroc    | BinaryAUROC    | 0      | train
4 | layer1   | Linear         | 301    | train
5 | layer2   | Linear         | 391    | train
----------------------------------------------------
692       Trainable params
0         Non-trainable params
692       Total params
0.003     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[I 2024-10-07 05:36:33,529] Trial 19 pruned. Trial was pruned at epoch 0.


Number of finished trials: 20
Best trial:
  Value: 0.609000027179718
  Params:
    optimizer: AdamW
    loss: BCELoss
    learning_rate: 0.00022595413314394904


# Evaluation

Metrics:
- F1 Score
- AOC
- ~~<span style="color:blue">Averaging</span>~~

Error analysis:
- Confusion matrix

## Implementation

F1 and AOC are implemented in the model class and callable by using a `Trainer` and calling the `test` function.
This shows the metric which are defined in the `test` functon.
The final evaluation is done with the test dataset.

The best model is loaded from wandb artifact registry and used on validation and test dataset.

All relevant metric can then be inspected in the wandb dashboard.

The confusion matrix is created by predicting on the test dataset and then logging it with wandb.

## Analysis
The accuracy of the final model is 61%, which is less than the skewed distribution of 62.2% True lables.

Looking at the confusion matrix reveals the cause of this number.
It mostly predicts 1 (Passage answers question) with some random 0 (Not answered) predictions.

In [28]:
# check wandb for trial id
BEST_TRIAL = "ngussih1"
checkpoint_reference = f"yelin-zhang-hslu/{WANDB_PROJECT}/model-{BEST_TRIAL}:best"
api = wandb.Api()
artifact = api.artifact(checkpoint_reference).download()
classifier_final = QClassifier.load_from_checkpoint(Path(artifact) / "model.ckpt")
wandb.init(project=WANDB_PROJECT, id=BEST_TRIAL, resume="allow")

eval_logger = WandbLogger(project=WANDB_PROJECT, log_model="all")
eval_trainer = L.Trainer(
    logger=eval_logger,
    accelerator="gpu",
    devices=1,
)

test_loader = utils.data.DataLoader(test, batch_size=32)
eval_trainer.test(classifier_final, dataloaders=test_loader)
pred_batches = eval_trainer.predict(classifier_final, dataloaders=test_loader)
preds = np.array([])
for batch in pred_batches:
    preds = np.concatenate((preds, batch.squeeze().numpy()))
conf = wandb.plot.confusion_matrix(
    y_true=test["answer"],
    preds=np.round(preds),
    class_names=["Not answered", "Answered"],
)
wandb.log({"test_conf_matrix": conf})

wandb:   1 of 1 files downloaded.  


epoch,0
test_accuracy,0.61651
test_auc,0.547
test_f1,0.75109
train_accuracy,0.45455
train_auc,0.73333
train_f1,0.625
train_loss,0.70682
trainer/global_step,0
val_accuracy,0.609
val_auc,0.57995


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.6165137887001038
        test_auc            0.5469972491264343
         test_f1            0.7510947585105896
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Predicting: |          | 0/? [00:00<?, ?it/s]

# Interpretation

Compare results of final model to expectations.
Run evaluation of final model with test dataset split.
But only at the very end, otherwise data leakage can happen.

Expectation: 70% accuracy with test dataset.

## Results

The reason for the 70% expectation is that the model should be better than always predicting true.
If always true is predicted the accuracy would be 62.3%. Because the dataset is skewed.

The final performance on the test dataset which was achieved is around 60%, therefore the model does not always predict true and tried to incorperate False predictions.
I assume because my model is very small it cannot learn the intricacies of the passage. In my opinion it is a positive aspect that the model did not learn to only predict true.

## Reflexion
The part which needed to be changed most was the model definition. I assumed 2 layers meant that I can do a 3 dimensional input.

But after clarification with the prof it meant that only 2 layers can be used and therefore I had to make the input 2 dimensional to reach the output dimension of 1.

Adding correctness test to the model definition to check if training, validation and test work was also very useful.

The assumptions for training were good as it worked as expected.

After looking into the options for creating the embeddings again, fasttext might not have been the best choice. A few articles on comparison between word2vec and fasttext highlight that word2vec usually performs better than fasttest for this type of task.